<a href="https://colab.research.google.com/github/AlexKostjuk/dz_17/blob/master/dz_17.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:

import pandas as pd
from google.colab import files


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
frb = pd.read_csv('/content/drive/MyDrive/T10Y2Y.csv', delimiter=',')


In [8]:
frb.head()

,observation_date,T10Y2Y
0,1976-06-01,0.68
1,1976-06-02,0.71
2,1976-06-03,0.70
3,1976-06-04,0.77
4,1976-06-07,0.79


In [9]:
frb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12737 entries, 0 to 12736
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   observation_date  12737 non-null  object 
 1   T10Y2Y            12201 non-null  float64
dtypes: float64(1), object(1)
memory usage: 199.1+ KB


In [ ]:
# Встановлення бібліотек у Colab
!pip install darts fbprophet statsmodels pmdarima

# Імпорт бібліотек
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from darts import TimeSeries
from darts.models import ExponentialSmoothing
from fbprophet import Prophet
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split
from google.colab import files

# Завантаження датасету
uploaded = files.upload()
df = pd.read_csv(list(uploaded.keys())[0])  # Завантажуємо перший завантажений файл

# Підготовка даних
df['observation_date'] = pd.to_datetime(df['observation_date'])
df = df.dropna()  # Видаляємо пропущені значення

# Розділення на X і y у стилі твого прикладу
X = df.drop(columns=['observation_date']).values  # У твоєму датасеті лише одна незалежна змінна
y = df['T10Y2Y'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Створюємо DataFrame для тренувальних і тестових даних із датами
train_df = df.iloc[:len(X_train)].copy()
test_df = df.iloc[len(X_train):].copy()

# --- Модель 1: Darts (Exponential Smoothing) ---
series = TimeSeries.from_dataframe(df, 'observation_date', 'T10Y2Y')
train_series = TimeSeries.from_dataframe(train_df, 'observation_date', 'T10Y2Y')
test_series = TimeSeries.from_dataframe(test_df, 'observation_date', 'T10Y2Y')

darts_model = ExponentialSmoothing()
darts_model.fit(train_series)
darts_pred = darts_model.predict(len(test_series))

# --- Модель 2: Prophet ---
prophet_df = train_df.rename(columns={'observation_date': 'ds', 'T10Y2Y': 'y'})
prophet_model = Prophet(daily_seasonality=True, yearly_seasonality=True)
prophet_model.fit(prophet_df)

future_dates = prophet_model.make_future_dataframe(periods=len(test_df))
prophet_pred = prophet_model.predict(future_dates)
prophet_pred_test = prophet_pred.tail(len(test_df))['yhat']

# --- Модель 3: SARIMA ---
sarima_model = SARIMAX(y_train,
                      order=(1, 1, 1),
                      seasonal_order=(1, 1, 1, 12),
                      enforce_stationarity=False,
                      enforce_invertibility=False)
sarima_results = sarima_model.fit()
sarima_pred = sarima_results.forecast(steps=len(y_test))

# Оцінка моделей
darts_pred_values = darts_pred.values().flatten()
prophet_pred_values = prophet_pred_test.values
sarima_pred_values = sarima_pred.values

metrics = {
    'Model': ['Darts', 'Prophet', 'SARIMA'],
    'MAE': [
        mean_absolute_error(y_test, darts_pred_values),
        mean_absolute_error(y_test, prophet_pred_values),
        mean_absolute_error(y_test, sarima_pred_values)
    ],
    'RMSE': [
        np.sqrt(mean_squared_error(y_test, darts_pred_values)),
        np.sqrt(mean_squared_error(y_test, prophet_pred_values)),
        np.sqrt(mean_squared_error(y_test, sarima_pred_values))
    ]
}

# Виводимо результати
results_df = pd.DataFrame(metrics)
print(results_df)

# Візуалізація
plt.figure(figsize=(12, 6))
plt.plot(test_df['observation_date'], y_test, label='Actual', color='black')
plt.plot(test_df['observation_date'], darts_pred_values, label='Darts', color='blue')
plt.plot(test_df['observation_date'], prophet_pred_values, label='Prophet', color='green')
plt.plot(test_df['observation_date'], sarima_pred_values, label='SARIMA', color='red')
plt.legend()
plt.title('Порівняння прогнозів моделей')
plt.xlabel('Дата')
plt.ylabel('T10Y2Y')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()